In [0]:
import pandas as pd
import numpy as np

In [0]:
import os
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Concatenate
from keras.layers import Conv1D, Conv2D, MaxPooling2D, GlobalMaxPooling1D, MaxPool1D, MaxPooling1D, SpatialDropout1D, GRU, Bidirectional, AveragePooling1D, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Input
from keras.layers.merge import concatenate
from keras.utils import to_categorical
from keras import layers
import tensorflow as tf
from tensorflow.keras import regularizers

Using TensorFlow backend.


In [0]:
import keras

In [0]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, hamming_loss, precision_recall_curve

In [0]:
padded_docs_train = np.load('X_train.npy')
padded_docs_valid = np.load('X_valid.npy')
embedding_matrix_fastext = np.load('embedding_matrix_fastext_ppd.npy')
embedding_matrix_glove = np.load('embedding_matrix_glove_300.npy')
y_train_ohe = np.load('y_train.npy')
y_valid_ohe = np.load('y_valid.npy')

In [0]:
max_features = embedding_matrix_fastext.shape[0] ## vocabulary size
maxlen = padded_docs_train.shape[1]
embed_size = 300 ## for glove using 300d dataset
n_class = 12

epochs = 12
batch_size = 100

In [0]:
def define_model(length, max_features):
  # channel 1
  inputs1 = Input(shape=(length,))
  embedding1 = Embedding(max_features, 300, weights=[embedding_matrix_fastext], trainable=False)(inputs1)

  bi_gru = Bidirectional(GRU(278, return_sequences=True))(embedding1)

  global_pool = GlobalMaxPooling1D()(bi_gru)
  avg_pool = GlobalAveragePooling1D()(bi_gru)

  concat_layer = Concatenate()([global_pool, avg_pool])

  output = Dense(n_class, activation='sigmoid')(concat_layer)

  model=Model(inputs1, output)

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_accuracy'])
  model.summary()
  return model

In [0]:
model_gru_4 = define_model(maxlen, max_features)

In [0]:
model_gru_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru_4.fit(padded_docs_train, y_train_ohe, validation_split=0.1, epochs=epochs, batch_size=batch_size, verbose=1)

In [0]:
pred_bidgru_fastext_4 = model_gru_4.predict(padded_docs_valid)

In [0]:
# PRECISION & RECALL
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, hamming_loss
predictions_results = []

thresholds=[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for val in thresholds:
  pred=pred_bidgru_fastext_4.copy()
  pred[pred>=val]=1
  pred[pred<val]=0
  
  accuracy = accuracy_score(y_valid_ohe, pred, normalize=True, sample_weight=None)
  hamming = hamming_loss(y_valid_ohe, pred)
  precision = precision_score(y_valid_ohe, pred, average='micro')
  recall = recall_score(y_valid_ohe, pred, average='micro')
  f1 = f1_score(y_valid_ohe, pred, average='micro')
  case= {'Threshold': val,
          'Accuracy': accuracy,
          'Hamming loss': hamming,
          'Precision': precision,
          'Recall': recall,
          'F1-measure': f1}
  predictions_results.append(case)

print("Micro-average quality numbers:")
pd.DataFrame(predictions_results)